In [54]:
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiDiscrete, flatdim
import numpy as np
import sys
import pandas as pd
import random
from collections import OrderedDict

In [256]:
class VPPBiddingEnv(Env):
    
    def __init__(self,
                 renewables_df, 
                 bids_df,
                 market_result_df,
                 tenders_df,
                 hist_window_size,
                 forecast_window_size,
                 frame_bound):
        
        # data 
        self.renewables_df = renewables_df
        self.market_result_df = market_result_df
        self.tenders_df = tenders_df
        self.bids_df = bids_df
        
        # window_size
        self.hist_window_size = hist_window_size
        self.forecast_window_size = forecast_window_size
        self.frame_bound = frame_bound
        
        self.hydro_df, self.wind_df = self._process_data()
        
        # episode
        self._start_date = pd.to_datetime(self.frame_bound[0])
        self._end_date = pd.to_datetime(self.frame_bound[1])
        self._done = None
        self._current_date = None
        self._total_reward = None
        self._total_profit = None
        self.history = None
        
        # spaces
        
        low = np.array([0.0] * 96) #96 timesteps to min 0.0
        high = np.array([1.0] * 96) #96 timesteps to max 1.0

        self.nested_observation_space = Dict({
            'historic_data': Dict({
                "hydro_historic": Box(low, high, dtype=np.float32),
                "wind_historic":  Box(low, high, dtype=np.float32)
            }),
            'forecast_data':  Dict({
                "hydro_forecast": Box(low, high, dtype=np.float32),
                "wind_forecast": Box(low, high, dtype=np.float32),
                "soc_forecast": Box(low, high, dtype=np.float32)
                # TODO should I keep the Battery state of charge? 
            }),
            'market_data':  Dict({
                "market_demand": Discrete(3), # for the demands 573, 562 and 555 MW
                # TODO for 2021 its always 562, how to handle differetn years? maybe set it as a global constant? 

                "predicted_market_prices":  Box(low=0.0, high=1634.52, shape=(6, 1), dtype=np.float32), # for each slot, can be prices of same day last week 
            }),
            'time_features':  Dict({
                "weekday": Discrete(7), # for the days of the week
                "holiday": Discrete(2), # holiday = 1, no holiday = 0
                "month": Discrete(12), # for the month
            })
        })
        
        # first approach: 
        # Slots 1,2,3,4,5,6 = integrated in bidsize if 0 or non 0 
        # bid size: MultiDiscrete([ 25, 25, 25, 25, 25 ]),
        # bid prize: Box(low=0.0, high=1634.52, shape=(6,), dtype=np.float32))

        self.action_space = Tuple((
            # returns array([ 0,  2, 13,  6, 23, 25 ]
            MultiDiscrete([ 25, 25, 25, 25, 25 , 25 ]),
            # returns array([1311.5632  ,  665.4643  ,  807.9639  ,  104.337715,  425.967, 205.23262 ]
            Box(low=0.0, high=1634.52, shape=(6,), dtype=np.float32)))
        
        # TODO: Add second approach with shares of plants
        '''
        second approach: 
            Share of hydro
            Share of wind
            Share of battery
        '''
    
    def reset(self):
        self._done = False
        self._current_date = self._start_date
        
        self._get_new_timestamps()

        self._total_reward = 0.
        self._total_profit = 0. 
        self.history = {}
        
        return self._get_observation()
    
    def _get_new_timestamps(self):
        
        self._historic_data_start = self._current_date - pd.offsets.DateOffset(days=1)
        self._historic_data_end =  self._current_date
        
        self._forecast_start = self._current_date + pd.offsets.DateOffset(days=1) # TODO: validate 
        self._forecast_end = self._current_date + pd.offsets.DateOffset(days=2) # TODO: validate
        
        self._market_start = self._current_date
        self._market_end = self._current_date + pd.offsets.DateOffset(hours=20)
        
        self._slot_date_list = []
        
        slot_date = self._current_date
        
        for i in range(0,6):
            self._slot_date_list.append(str(slot_date))
            slot_date = slot_date + pd.offsets.DateOffset(hours=4)        
    
    def _get_observation(self):
        hydro_historic = self.hydro_df[str(self._historic_data_start) : str(self._historic_data_end)].to_numpy()
        wind_historic = self.wind_df[str(self._historic_data_start) : str(self._historic_data_end)].to_numpy()
        
        # TODO: change current date start to 07:00 of the current day, to be shortly before auction time
        hydro_forecast = self.hydro_df[str(self._historic_data_start) : str(self._historic_data_end)].to_numpy()
        wind_forecast =  self.wind_df[str(self._forecast_start) : str(self._forecast_end)].to_numpy()
        
        market_demand = 0  # TODO: validate if market demand can be induce in another way, maybe on function call ? 
        predicted_market_prices = [10. , 20. , 10. , 20. , 10. , 20.] # TODO: naive prediction: retrieve price of same day last week 
        weekday = random.randrange(7) # TODO: retrieve from time dataframe 
        holiday = random.randrange(2) # TODO: retrieve from time dataframe 
        month = random.randrange(12) # TODO: retrieve from time dataframe 

        
        next_observation = OrderedDict({
            "historic_data": OrderedDict({
                "hydro_historic": hydro_historic,
                "wind_historic": wind_historic
                }),
            "forecast_data": OrderedDict({
                "hydro_forecast": hydro_forecast,
                "wind_forecast": wind_forecast
            }),
            "market_data": OrderedDict({
                "market_demand": market_demand,
                "predicted_market_prices": predicted_market_prices
            }),
            "time_features": OrderedDict({
                "weekday": weekday, 
                "holiday": holiday, 
                "month": month
            }), 
            
        })
                
        # ----------------------
        
        '''
        next_observation = Dict({
            'historic_data': Dict({
                "hydro_historic": Box(low, high, dtype=np.float32)
                "wind_historic":  Box(low, high, dtype=np.float32)
            }),
            'forecast_data':  Dict({
                "hydro_forecast": Box(low, high, dtype=np.float32),
                "wind_forecast": Box(low, high, dtype=np.float32),
                "soc_forecast": Box(low, high, dtype=np.float32)
                # TODO should I keep the Battery state of charge? 
            }),
            'market_data':  Dict({
                "market_demand": Discrete(3), # for the demands 573, 562 and 555 MW
                # TODO for 2021 its always 562, how to handle differetn years? maybe set it as a global constant? 

                "predicted_market_prices":  Box(low=0.0, high=1634.52, shape=(6, 1), dtype=np.float32), # for each slot, can be prices of same day last week 
            }),
            'time_features':  Dict({
                "weekday": Discrete(7), # for the days of the week
                "holiday": Discrete(2), # holiday = 1, no holiday = 0
                "month": Discrete(12), # for the month
            })
        })
        '''
            
        return next_observation
    
    def step(self, action):
        
        self._simulate_market(action)
        
        # calculate reward from state and action 
        step_reward = self._calculate_reward(action)
        self._total_reward += step_reward
       
        self._update_profit(action)
        
        self._get_new_timestamps()
        
        observation = self._get_observation()
                
        info = dict(
            current_date = str(self._current_date),
            total_reward = self._total_reward,
            total_profit = self._total_profit
        )
        
        self._update_history(info)
        # TODO: info can contain state variables that are hidden from observations
        # or individual reward terms that are combined to produce the total reward
        
        self._current_date =  self._current_date + pd.offsets.DateOffset(days=1)
            
        self._done = True
        
        return observation, step_reward, self._done, info
    
    def _calculate_reward(self, action):
        # TODO: Create Market Clearing algorithm (first easy version: check if price is < Settlement price, 
        # pro-version would be: recreate merit order: to see if the offerd Capacity (menge) would be in won offers. )
        
        # TODO: validate if capacity could be provided
        
        # TODO: write reward function 
        step_reward = 1.0
        return step_reward
    
    def _update_profit(self, action):
        trade = False 
        if trade == False: 
            self._total_profit +=0
        if trade == True: 
            self._total_profit = 1
        # TODO: implement profit function 

    
    def _update_history(self, info):
        if not self.history:
            self.history = {key: [] for key in info.keys()}

        for key, value in info.items():
            self.history[key].append(value)

    def render(self):
        # Implement visulisation
        pass
    
    def _process_data(self):
        hydro_df = self.renewables_df.loc[:, 'Hydro1']
        wind_df = self.renewables_df.loc[:, 'WP1']
        # TODO: add more power plants
        return hydro_df, wind_df

    def _simulate_market(self, action): 
        
        current_bids = self.bids_df[self._market_start : self._market_end]
        
        for i in range(0, len(self._slot_date_list)):
            slot_date = self._slot_date_list[i]
            print("slot_date = %s" % (slot_date))

            bids_in_slot = current_bids[slot_date : slot_date].reset_index(drop=True).reset_index(drop=False)
            
            bids_list = bids_slot_1.to_dict('records')
            agents_bid_index = len(bids_list)
            agents_bid_size = action[0][i]
            agents_bid_price = action[1][i]
            print("agents_bid_size = %s" % (agents_bid_size))
            print("agents_bid_price = %s" % (agents_bid_price))

            bids_list.append({'index': agents_bid_index, 'capacity': agents_bid_size, 'price': agents_bid_price})

            sorted_bids_list_by_price = sorted(bids_list, key=lambda x: x['price'])

            accumulated_capacity = 0
            total_demand = 500
            last_price = 0
            agents_bid_in_result = False

            for bid in sorted_bids_list_by_price:
                accumulated_capacity += bid["capacity"]
                if bid["index"] == agents_bid_index: 
                        agents_bid_in_result = True
                if accumulated_capacity > total_demand:
                    last_price =  bid["price"]
                    print("accumulated_capacity = %s" % (accumulated_capacity))
                    print("last_price = %s" % (last_price))
                    print("agents_bid_in_result = %s" % (agents_bid_in_result))
                    break

        # for each slot 
            # get all bids
            # bids to dict
            # add bid from action 
            # bring in order by price 
            # accumulate capacities until demand is filled 
            # check if bid is in bid list 
                # if yes, set auciton_won = True and get SETTLEMENTCAPACITY_PRICE
                # if no, set auciton_won = False

In [258]:
# TODO: change Aufbau of offer Dataframe 
# aufbau df 
# date # slot = 1,2,3,4,5,6

In [259]:
renewables_df = pd.read_csv("data/clean/renewables.csv", sep = ";").set_index("time", drop = True)

bids_df = pd.read_csv("data/clean/bids_all.csv", sep = ";", index_col = 0).set_index("SLOT_START", drop = True)
bids_df.index = pd.to_datetime(bids_df.index)
bids_df = bids_df.rename(columns={'OFFERED_CAPACITY_PRICE_[EUR/MW]': 'price', 'OFFERED_CAPACITY_[MW]': 'capacity'})
bids_df = bids_df[["capacity", "price"]]

market_result_df = []
tenders_df = []

hist_window_size = 1 # in days
forecast_window_size = 1 # in days
start_index = "2021-01-02 00:00:00+00:00"
end_index = "2021-12-30 00:00:00+00:00"
frame_bound = (start_index, end_index)

env = VPPBiddingEnv(renewables_df,
                    bids_df,
                    market_result_df, 
                   tenders_df, 
                   hist_window_size, 
                   forecast_window_size, 
                   frame_bound)

/Users/Jan-Lukas.Pflaum/.virtualenvs/thesis/lib/python3.8/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [260]:
#episodes = 365
episodes = 1
score = 0 
observation = env.reset()

for episode in range(1, episodes+1):    
    #env.render()
    action = env.action_space.sample()
    n_observation, reward, done, info = env.step(action)
    score+=reward
    print('Episode:{} Score:{} Info:{}'.format(episode, score, info))

['2021-01-02 00:00:00+00:00', '2021-01-02 04:00:00+00:00', '2021-01-02 08:00:00+00:00', '2021-01-02 12:00:00+00:00', '2021-01-02 16:00:00+00:00', '2021-01-02 20:00:00+00:00']
                           capacity  price
SLOT_START                                
2021-01-02 00:00:00+00:00      11.0    0.0
2021-01-02 00:00:00+00:00       1.0    0.0
2021-01-02 00:00:00+00:00       4.0    0.0
2021-01-02 00:00:00+00:00       7.0    0.0
2021-01-02 00:00:00+00:00       1.0    0.0
...                             ...    ...
2021-01-02 20:00:00+00:00      10.0   23.4
2021-01-02 20:00:00+00:00       5.0   25.0
2021-01-02 20:00:00+00:00       8.0   25.0
2021-01-02 20:00:00+00:00       5.0   25.0
2021-01-02 20:00:00+00:00       5.0   25.0

[440 rows x 2 columns]
slot_date = 2021-01-02 00:00:00+00:00
agents_bid_size = 1
agents_bid_price = 953.18506
accumulated_capacity = 502.0
last_price = 10.0
agents_bid_in_result = False
slot_date = 2021-01-02 04:00:00+00:00
agents_bid_size = 6
agents_bid_price = 87

In [125]:
bids_df[:"2021-01-02 20:00:00+00:00"]

,OFFERED_CAPACITY_PRICE_[EUR/MW],OFFERED_CAPACITY_[MW],ALLOCATED_CAPACITY_[MW],SETTLEMENTCAPACITY_PRICE_[EUR/MW],NOTE
SLOT_START,,,,,
2020-07-01 00:00:00+00:00,0.0,1.0,1.0,31.46,NaN
2020-07-01 00:00:00+00:00,0.0,2.0,2.0,31.46,INDIVISIBLE
2020-07-01 00:00:00+00:00,0.0,2.0,2.0,31.46,NaN
2020-07-01 00:00:00+00:00,0.0,10.0,10.0,31.46,INDIVISIBLE
2020-07-01 00:00:00+00:00,0.0,6.0,6.0,31.46,INDIVISIBLE
...,...,...,...,...,...
2021-01-02 20:00:00+00:00,23.4,10.0,10.0,25.00,NaN
2021-01-02 20:00:00+00:00,25.0,5.0,3.0,25.00,NaN
2021-01-02 20:00:00+00:00,25.0,8.0,8.0,25.00,NaN


In [32]:
display(env.nested_observation_space.sample())

OrderedDict([('forecast_data',
              OrderedDict([('hydro_forecast',
                            array([0.3723876 , 0.55196923, 0.95489025, 0.541279  , 0.7360674 ,
                                   0.95782   , 0.21991904, 0.3673116 , 0.45163104, 0.2437514 ,
                                   0.95953226, 0.61918455, 0.95530474, 0.85252106, 0.892614  ,
                                   0.72320896, 0.8237727 , 0.6337547 , 0.18438564, 0.18405148,
                                   0.24572708, 0.8979379 , 0.6744241 , 0.8459417 , 0.66005725,
                                   0.5348232 , 0.9653753 , 0.39756793, 0.8563647 , 0.3855034 ,
                                   0.778922  , 0.5708247 , 0.6829965 , 0.8306585 , 0.10521574,
                                   0.5106071 , 0.13330176, 0.57339436, 0.12440432, 0.79017407,
                                   0.6572475 , 0.9642127 , 0.70960265, 0.5744724 , 0.7899647 ,
                                   0.04629783, 0.32578138, 0.1918089

In [76]:
env.nested_observation_space

Dict(forecast_data:Dict(hydro_forecast:Box(0.0, 1.0, (96,), float32), soc_forecast:Box(0.0, 1.0, (96,), float32), wind_forecast:Box(0.0, 1.0, (96,), float32)), historic_data:Dict(hydro_historic:Box(0.0, 1.0, (96,), float32), wind_historic:Box(0.0, 1.0, (96,), float32)), market_data:Dict(market_demand:Discrete(3), predicted_market_prices:Box(0.0, 1634.52, (6, 1), float32)), time_features:Dict(holiday:Discrete(2), month:Discrete(12), weekday:Discrete(7)))

In [77]:
flatdim(env.nested_observation_space)

510

In [84]:
from gym.spaces import flatten_space
flatten_space(env.nested_observation_space)

Box(0.0, [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00

In [86]:
# flatten(space, x)
# Flatten a data point from a space.
# This is useful when e.g. points from spaces must be passed to a neural network, which only understands flat arrays of floats.
# Accepts a space and a point from that space. Always returns a 1D array. 

from gym.spaces import flatten
flatten(env.nested_observation_space, env.nested_observation_space.sample())

array([8.11625838e-01, 1.92382410e-01, 6.88461483e-01, 9.52754617e-02,
       3.76577318e-01, 8.39748621e-01, 4.96831715e-01, 4.27501947e-01,
       3.46993774e-01, 3.70248139e-01, 3.30729008e-01, 6.26657084e-02,
       9.59646702e-01, 9.56731260e-01, 5.43962955e-01, 7.07620084e-01,
       6.91270009e-02, 1.92220062e-01, 5.35759151e-01, 3.80583018e-01,
       1.96395338e-01, 8.10885206e-02, 3.43103170e-01, 1.72102049e-01,
       5.34813404e-01, 8.82097483e-01, 8.54062855e-01, 5.35712063e-01,
       1.47296816e-01, 8.32087874e-01, 9.60846066e-01, 7.68246174e-01,
       7.13113189e-01, 4.42189068e-01, 4.46793407e-01, 3.31610963e-02,
       8.59159887e-01, 4.29612845e-01, 6.42166793e-01, 3.23513836e-01,
       6.66281462e-01, 1.45036727e-01, 2.37793222e-01, 3.83083880e-01,
       2.65617430e-01, 5.11561811e-01, 8.83708715e-01, 2.18019839e-02,
       1.62229970e-01, 6.73336446e-01, 8.85052502e-01, 5.74002624e-01,
       3.32120880e-02, 5.51064432e-01, 6.00537598e-01, 1.58330649e-01,
      

In [85]:
# check if flattened data point is in space

flatten((env.nested_observation_space, env.nested_observation_space.sample()) in flatten_space(env.nested_observation_space)

True

In [89]:
from gym.spaces import unflatten

flattened_datapoint = flatten(env.nested_observation_space, env.nested_observation_space.sample())
unflattened_datapoint = unflatten(env.nested_observation_space, flattened_datapoint)
unflattened_datapoint

OrderedDict([('forecast_data',
              OrderedDict([('hydro_forecast',
                            array([5.5335385e-01, 2.2821391e-01, 2.5449359e-01, 5.5017620e-01,
                                   2.2021979e-01, 7.3971874e-01, 7.9568899e-01, 2.7310282e-01,
                                   6.6302556e-01, 6.2482554e-01, 3.8765112e-01, 8.9774579e-01,
                                   3.4189326e-01, 9.3739784e-01, 1.0861359e-01, 6.7293733e-01,
                                   2.0193619e-01, 4.5053428e-01, 6.4418912e-01, 7.0260507e-01,
                                   9.4499692e-02, 4.9046814e-02, 4.4654372e-01, 8.2932329e-01,
                                   6.5390772e-01, 7.8328915e-02, 4.5716800e-02, 9.0136534e-01,
                                   6.7176574e-01, 3.7618262e-01, 8.1752196e-02, 1.7736143e-02,
                                   9.9274075e-01, 1.0689364e-01, 5.4141659e-01, 6.2379336e-01,
                                   8.1284511e-01, 5.7165284e-04, 5.4

In [244]:
action = env.action_space.sample()
display(action)

bid_size = action[0][4]
bid_price = action[1][0]

display(bid_size)
display(bid_price)

(array([ 4,  5,  9, 11, 24, 14]),
 array([1540.1139 ,  571.4528 , 1133.2961 ,  123.10094,  470.4612 ,
        1498.1427 ], dtype=float32))

24

1540.1139

# 2. Create a Deep Learning Model with Keras

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [157]:
states_hydro_historic = env.nested_observation_space["historic_data"]["hydro_historic"].shape
states_wind_historic = env.nested_observation_space["historic_data"]["wind_historic"].shape

actions = env.action_space[0].shape

In [158]:
display(states_hydro_historic)
display(states_wind_historic)
display(actions)

(96,)

(96,)

(6,)

In [ ]:
def build_model(states, actions):
    model = Sequential()
    # flatten? 
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
del model 


In [ ]:
model = build_model(states, actions)


In [ ]:
model.summary()


# 3. Build Agent with Keras-RL


In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory


In [ ]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

In [ ]:
_ = dqn.test(env, nb_episodes=15, visualize=True)


# 4. Reloading Agent from Memory


In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)


In [ ]:
del model
del dqn
del env

In [ ]:
from gym.envs.registration import register

register(
    id='vpp-v0',
    entry_point='gym_foo.envs:FooEnv',
)


In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('dqn_weights.h5f')


In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)
